In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import time
from lightgbm import LGBMClassifier
import lightgbm as lgb
from xgboost import XGBClassifier

import warnings
warnings.simplefilter('ignore', UserWarning)

import gc
gc.enable()

In [ ]:
input_path=""
submission_path=""
feature_path=""
print("Load numerical features")
df_num = pd.concat([
            pd.read_feather(feature_path+"feature_num_features.ftr"),
            #pd.read_feather(feature_path+"feature_arithmetic+_combi2.ftr"),
            #pd.read_feather(feature_path+"feature_arithmetic-_combi2.ftr"),
            pd.read_feather(feature_path+"feature_arithmetic*_combi2.ftr"),
            pd.read_feather(feature_path+"feature_arithmeticdiv_combi2.ftr"),
            pd.read_feather(feature_path+"train_test_7th_df.ftr"),
            pd.read_feather(feature_path"train_test_oliverFE_df.ftr")
            ],axis=1,)
y_train = df_num["TARGET"].dropna()
df_num.drop(["TARGET"], axis=1, inplace=True)

print("Load categorical features")
df = pd.concat([
            pd.read_feather(feature_path+"feature_1way_label_encoding_with_te.ftr"),
            pd.read_feather(feature_path+"feature_2way_label_encoding_with_te.ftr"),
            #pd.read_feather(feature_path+"feature_3way_including_CODE_GENDER_label_encoding_with_te.ftr"),
            pd.read_feather(feature_path+"feature_round_num_label_encoding.ftr"),
            ],axis=1,)
df = pd.concat([df, df_num], axis=1)
X_train = df.iloc[:y_train.shape[0],:]
 X_test = df.iloc[y_train.shape[0]:,:]

In [ ]:
actual_imp_df=pd.read_csv(feature_path+"actual_imp_df.csv")
null_imp_df=pd.read_csv(feature_path+"null_imp_df.csv") 

THRESHOLD = 97
# 閾値を超える特徴量を取得
imp_features = []
for feature in actual_imp_df["feature"]:
    actual_value = actual_imp_df.query(f"feature=='{feature}'")["importance"].values
    null_value = null_imp_df.query(f"feature=='{feature}'")["importance"].values
    percentage = (null_value < actual_value).sum() / null_value.size * 100
    if percentage >= THRESHOLD:
        imp_features.append(feature)
len(imp_features)


In [ ]:
add_feature2=pd.read_feather(feature_path+"github_feature2.fhr")
train_df=pd.concat([X_train.loc[:, imp_features], add_feature2.iloc[:y_train.shape[0],:]], axis=1)
test_df=pd.concat([X_test.loc[:, imp_features], add_feature2.iloc[y_train.shape[0]:,:]], axis=1)
y_train=pd.read_csv(input_path+"train.csv").TARGET

In [ ]:
test_df=test_df.drop("Unnamed: 0", axis="columns")

In [ ]:
test_df.head()

In [ ]:
train_df.head()

In [ ]:
gc.collect()

In [ ]:
oof_preds = np.zeros(train_df.shape[0]) #num feature 226 thre=97
sub_preds = np.zeros(test_df.shape[0])

num_folds=5
folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, y_train)):
        train_x, train_y = train_df.iloc[train_idx], y_train.iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], y_train.iloc[valid_idx]


        clf = XGBClassifier(
            #nthread=4,
            n_estimators=10000,
            learning_rate=0.005134,
            num_leaves=54,
            colsample_bytree=0.508716,
            subsample=0.8715623,
            max_depth=10,
            reg_alpha=0.436193,
            reg_lambda=0.479169,
            min_split_gain=0.024766,
            min_child_weight=39.3259775,
            #silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        sub_preds += clf.predict_proba(test_df)[:, 1] / folds.n_splits


        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

input_path=""
submission_path=""
result=pd.read_csv(input_path+"sample_submission.csv")
result["TARGET"]=sub_preds
result.to_csv(submission_path+"226feat_5fold_seed1001_XGB.csv")